In [1]:
def prepare_playfair_key(key):
    # Convert to uppercase and remove spaces
    key = ''.join(key.upper().split())

    # Replace J with I
    key = key.replace('J', 'I')

    # Create the key square (5x5 grid)
    alphabet = "ABCDEFGHIKLMNOPQRSTUVWXYZ"  # Note: I and J share a position
    key_square = []

    # First fill in the key
    for char in key:
        if char in alphabet and char not in key_square:
            key_square.append(char)
            alphabet = alphabet.replace(char, '')

    # Then fill with remaining letters
    key_square.extend(alphabet)

    # Convert to 5x5 grid
    grid = [key_square[i:i+5] for i in range(0, 25, 5)]
    return grid

def find_position(grid, letter):
    # Find the row and column of a letter in the grid
    for i, row in enumerate(grid):
        if letter in row:
            j = row.index(letter)
            return i, j
    return -1, -1  # This should never happen if the input is valid

def prepare_text(text):
    # Convert to uppercase and remove spaces and punctuation
    text = ''.join(c for c in text.upper() if c.isalpha())

    # Replace J with I
    text = text.replace('J', 'I')

    # Handle pairs - separate identical letters with 'X'
    i = 0
    result = []
    while i < len(text):
        if i == len(text) - 1:
            result.append(text[i] + 'X')
            break
        elif text[i] == text[i+1]:
            result.append(text[i] + 'X')
            i += 1
        else:
            result.append(text[i] + text[i+1])
            i += 2

    return result

def encrypt_pair(grid, pair):
    # Get positions
    r1, c1 = find_position(grid, pair[0])
    r2, c2 = find_position(grid, pair[1])

    # Same row
    if r1 == r2:
        return grid[r1][(c1+1)%5] + grid[r2][(c2+1)%5]

    # Same column
    elif c1 == c2:
        return grid[(r1+1)%5][c1] + grid[(r2+1)%5][c2]

    # Rectangle
    else:
        return grid[r1][c2] + grid[r2][c1]

def decrypt_pair(grid, pair):
    # Get positions
    r1, c1 = find_position(grid, pair[0])
    r2, c2 = find_position(grid, pair[1])

    # Same row
    if r1 == r2:
        return grid[r1][(c1-1)%5] + grid[r2][(c2-1)%5]

    # Same column
    elif c1 == c2:
        return grid[(r1-1)%5][c1] + grid[(r2-1)%5][c2]

    # Rectangle
    else:
        return grid[r1][c2] + grid[r2][c1]

def encrypt_playfair(text, key):
    grid = prepare_playfair_key(key)
    pairs = prepare_text(text)
    result = []

    for pair in pairs:
        result.append(encrypt_pair(grid, pair))

    return ''.join(result)

def decrypt_playfair(text, key):
    grid = prepare_playfair_key(key)
    # Split the ciphertext into pairs
    pairs = [text[i:i+2] for i in range(0, len(text), 2)]
    result = []

    for pair in pairs:
        result.append(decrypt_pair(grid, pair))

    # Remove padding 'X's between repeated letters
    decrypted = ''.join(result)
    final = []
    i = 0
    while i < len(decrypted):
        final.append(decrypted[i])
        if i < len(decrypted) - 2 and decrypted[i] == decrypted[i+2] and decrypted[i+1] == 'X':
            i += 2
        else:
            i += 1

    # Remove trailing 'X' if present
    if final and final[-1] == 'X':
        final.pop()

    return ''.join(final)

# Example usage
if __name__ == "__main__":
    key = "PLAYFAIR EXAMPLE"
    plaintext = "HIDE THE GOLD IN THE TREE STUMP"

    encrypted = encrypt_playfair(plaintext, key)
    print(f"Encrypted: {encrypted}")

    decrypted = decrypt_playfair(encrypted, key)
    print(f"Decrypted: {decrypted}")

Encrypted: BMODZBXDNABEKUDMUIXMMOUVIF
Decrypted: HIDETHEGOLDINTHETREESTUMP
